In [1]:
import os, sys, torch, subprocess, prody, vmd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

self_dir = os.getcwd()
root_dir = os.path.normpath(self_dir + '/..' * 2)
package_dir = os.path.join(root_dir, 'src')
sys.path.append(package_dir)

# from ml_modules.data.datasets import Dataset
from ml_modules.data.enm import TNM_Computer
from ml_modules.data.retrievers import AlphaFold_Retriever



data_dir = '../../data/processed/v7a'

tnm_computer = TNM_Computer()
pdb_retriever = AlphaFold_Retriever()

coupling_types = ['codir', 'coord', 'deform']
edge_types = ['contact'] + coupling_types

vmd_resolution = 1024
v_cmap = plt.get_cmap('plasma')


setup = 'contact_12-codir_1CONT-coord_1CONT-deform_1CONT'



/Users/yenlin/miniconda3/envs/graphs/lib/python3.11/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Info) VMD for MACOSXX86_64, version 1.9.4a26 (January 29, 2025)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 4 CPUs detected.


In [2]:

stats_dir = 'stats'
os.makedirs(stats_dir, exist_ok=True)

plots_dir = 'plots'
os.makedirs(plots_dir, exist_ok=True)


et_color = {
    'contact': (0., 0.6039215686274509, 0.8705882352941177),
    'codir': (0., 0.803921568627451, 0.4235294117647059),
    'coord': (0.6862745098039216, 0.34509803921568627, 0.7294117647058823),
    'deform': (1., 0.7764705882352941, 0.11764705882352941),
}

labels = {
    et: f'{et} coupling' if et != 'contact' else 'distance (Å)'
    for et in edge_types
}

chain_color = {
    '0A': [0,0,255,255],
    '0B': [255,0,0,255],
    '0C': [0,255,255,255],
}

residue_color = {
    'ASP': [230,10,10,255],
    'GLU': [230,10,10,255],
    'CYS': [230,230,0,255],
    'MET': [230,230,0,255],
    'LYS': [20,90,255,255],
    'ARG': [20,90,255,255],
    'SER': [250,150,0,255],
    'THR': [250,150,0,255],
    'PHE': [50,50,170,255],
    'TYR': [50,50,170,255],
    'ASN': [0,220,220,255],
    'GLN': [0,220,220,255],
    'GLY': [235,235,235,255],
    'LEU': [15,130,15,255],
    'VAL': [15,130,15,255],
    'ILE': [15,130,15,255],
    'ALA': [200,200,200,255],
    'TRP': [180,90,180,255],
    'HIS': [130,130,210,255],
    # 'HSE': [130,130,210,255],
    'PRO': [220,150,130,255],
}

res3_to_res1 = {
    'CYS': 'C',
    'ASP': 'D',
    'SER': 'S',
    'GLN': 'Q',
    'LYS': 'K',
    'ILE': 'I',
    'PRO': 'P',
    'THR': 'T',
    'PHE': 'F',
    'ASN': 'N',
    'GLY': 'G',
    'HIS': 'H',
    #  'HSE': 'H',
    'LEU': 'L',
    'ARG': 'R',
    'TRP': 'W',
    'ALA': 'A',
    'VAL': 'V',
    'GLU': 'E',
    'TYR': 'Y',
    'MET': 'M',
}


In [3]:


stats_save_dir = f'{stats_dir}/{setup}'
files = os.listdir(stats_save_dir)
files.sort()

all_accessions = [
    f.split(' - ')[-1][:-4] for f in files
]
print(all_accessions[:5])

accessions_to_process = [
    'O31775-AFv4',
    'Q72JK8-AFv4',
    'R4YR45-AFv4',
]
print(accessions_to_process)


['A1Z7A8', 'A2BE93', 'A7MCK9', 'B2GTW6', 'G5EFV5']
['O31775-AFv4', 'Q72JK8-AFv4', 'R4YR45-AFv4']


In [4]:

# src_file = f'../20250526-1 true and baseline vs dynamics performance/stats/accessions - largest improvement (top 20).txt'

# with open(os.path.abspath(src_file), 'r') as f:
#     lines = [l.replace('\n', '') for l in f.readlines()]

# lines_to_keep = []
# for line_idx, line in enumerate(lines):
#     if 'dynamics 4' in line:
#         lines_to_keep.append(lines[line_idx+1:line_idx+21])

# proteins_in_grp = []
# for line_grp in lines_to_keep:
#     proteins_in_grp.append(
#         [entry.split()[0][:-1] for entry in line_grp]
#     )

# uni, cnt = np.unique(proteins_in_grp, return_counts=True)
# accessions_to_process = uni[cnt == 1]

# print(accessions_to_process)


In [5]:

stats_save_dir = f'{stats_dir}/{setup}'
os.makedirs(stats_save_dir, exist_ok=True)

for acc_idx, accession in enumerate(accessions_to_process):

    plots_save_dir = f'{plots_dir}/{setup}/on structure/{accession}'
    os.makedirs(plots_save_dir, exist_ok=True)

    pdb_file = os.path.abspath(pdb_retriever.path_to_file(accession))
    assert os.path.exists(pdb_file)

    graph_file = os.path.abspath(f'{data_dir}/graphs/{setup}/{accession}.pt')
    data = torch.load(graph_file)

    atoms = prody.parsePDB(pdb_file, subset='ca')
    resnames1 = [
        res3_to_res1[resname] for resname in atoms.getResnames()
    ]

    n_nodes = data.num_nodes
    node_list = np.arange(n_nodes, dtype=np.int_)

    graphs = {}
    for et in edge_types:

        graphs[et] = nx.Graph()
        graphs[et].add_nodes_from(node_list)

        edge_index = data['residue', et, 'residue'].edge_index
        # remove reverse edges for undirected graphs
        edge_index = edge_index[:, edge_index[0] < edge_index[1]]

        graphs[et].add_edges_from(edge_index.T.tolist())

        assert graphs[et].number_of_nodes() == n_nodes
        assert graphs[et].number_of_edges() == edge_index.shape[1]

        ################################################################
        # COMPUTE DEGREE AND PLOT WITH VMD
        ################################################################
        output_filename = f'{accession} - {et} - degree.png'

        degree = np.array(graphs[et].degree())[:,1]

        nx.set_node_attributes(
            graphs[et],
            dict(graphs[et].degree()),
            'degree'
        )

        ### LOAD STRUCTURE
        molid = vmd.molecule.load(
            struct_type='pdb',
            struct_file=pdb_file,
            coord_type='pdb',
            coord_file=pdb_file,
        )

        ### SET DISPLAY
        vmd.display.set(
            antialias=True,
            depthcue=False,
            culling=True,
            ambientocclusion=True,
            projection='Orthographic',
            size=(vmd_resolution, vmd_resolution),
            aoambient=0.75,
            aodirect=0.75,
            shadows=False,
            dof=False
        )
        vmd.axes.set_location('OFF')

        ### DRAW VERTICES (RESIDUES)
        backbone = vmd.atomsel('name CA')
        coords = np.vstack([
            backbone.x,
            backbone.y,
            backbone.z,
        ]).T

        n_vertex_colors = 15
        bin_number = np.digitize(
            degree,
            np.linspace(
                0,
                degree.max() if degree.max() > 0 else 1,
                n_vertex_colors,
                endpoint=False
            )
        )
        bin_color = v_cmap( np.linspace(0,1,n_vertex_colors+1)[1:] )[:,:3]

        for color_idx in range(n_vertex_colors):
            bin_idx = color_idx + 1
            coords_subset = coords[bin_number == bin_idx]
            vmd.color.set_colorid(color_idx, tuple(bin_color[color_idx]))
            vmd.graphics.color(molid, color_idx)

            for coord in coords_subset:
                vmd.graphics.sphere(molid, coord, radius=1, resolution=6)

        ### MODIFY MOLECULE REPRESENTATION
        next_color_idx = 0 + n_vertex_colors
        vmd.color.set_colorid(next_color_idx, (1.,1.,1.))
        vmd.graphics.color(molid, next_color_idx)
        if not vmd.molrep.modrep(
            molid,
            0,
            style='NewCartoon',
            color=f'ColorID {next_color_idx}',
            # selection='backbone',
            material='Opaque',
        ): continue

        # vmd.trans.rotate_scene(axis='z', angle=180)
        # vmd.trans.rotate_scene(axis='y', angle=180)

        ### RENDER
        vmd.display.update()
        vmd.render.render(
            method='POV3',
            filename='tmp.pov',
        )

        cmd = [
            'povray',
            f'+W{vmd_resolution}',
            f'+H{vmd_resolution}',
            f'-Itmp.pov',
            f'-O"{plots_save_dir}/{output_filename}"',
            '+X',
            '+A',
            '+FN',
            '+UA',
        ]
        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )

        ### REMOVE MOLECULE FROM DISPLAY
        vmd.molecule.delete(molid)

        ################################################################
        # COMPUTE BETWEENNESS AND PLOT WITH VMD
        ################################################################
        output_filename = f'{accession} - {et} - betweenness.png'

        betweenness = np.array(list(
            nx.betweenness_centrality(graphs[et]).items()
        ))[:,1]

        nx.set_node_attributes(
            graphs[et],
            dict(nx.betweenness_centrality(graphs[et])),
            'betweenness'
        )

        ### LOAD STRUCTURE
        molid = vmd.molecule.load(
            struct_type='pdb',
            struct_file=pdb_file,
            coord_type='pdb',
            coord_file=pdb_file,
        )

        ### SET DISPLAY
        vmd.display.set(
            antialias=True,
            depthcue=False,
            culling=True,
            ambientocclusion=True,
            projection='Orthographic',
            size=(vmd_resolution, vmd_resolution),
            aoambient=0.75,
            aodirect=0.75,
            shadows=False,
            dof=False
        )
        vmd.axes.set_location('OFF')

        ### DRAW VERTICES (RESIDUES)
        backbone = vmd.atomsel('name CA')
        coords = np.vstack([
            backbone.x,
            backbone.y,
            backbone.z,
        ]).T

        n_vertex_colors = 15
        bin_number = np.digitize(
            betweenness,
            np.linspace(
                0,
                betweenness.max() if betweenness.max() > 0 else 1,
                n_vertex_colors,
                endpoint=False
            )
        )
        bin_color = v_cmap( np.linspace(0,1,n_vertex_colors+1)[1:] )[:,:3]

        for color_idx in range(n_vertex_colors):
            bin_idx = color_idx + 1
            coords_subset = coords[bin_number == bin_idx]
            vmd.color.set_colorid(color_idx, tuple(bin_color[color_idx]))
            vmd.graphics.color(molid, color_idx)

            for coord in coords_subset:
                vmd.graphics.sphere(molid, coord, radius=1, resolution=6)

        ### MODIFY MOLECULE REPRESENTATION
        next_color_idx = 0 + n_vertex_colors
        vmd.color.set_colorid(next_color_idx, (1.,1.,1.))
        vmd.graphics.color(molid, next_color_idx)
        if not vmd.molrep.modrep(
            molid,
            0,
            style='NewCartoon',
            color=f'ColorID {next_color_idx}',
            # selection='backbone',
            material='Opaque',
        ): continue

        # vmd.trans.rotate_scene(axis='z', angle=180)
        # vmd.trans.rotate_scene(axis='y', angle=180)

        ### RENDER
        vmd.display.update()
        vmd.render.render(
            method='POV3',
            filename='tmp.pov',
        )

        cmd = [
            'povray',
            f'+W{vmd_resolution}',
            f'+H{vmd_resolution}',
            f'-Itmp.pov',
            f'-O"{plots_save_dir}/{output_filename}"',
            '+X',
            '+A',
            '+FN',
            '+UA',
        ]
        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )

        ### REMOVE MOLECULE FROM DISPLAY
        vmd.molecule.delete(molid)

        ################################################################
        # COMPUTE CLOSENESS AND PLOT WITH VMD
        ################################################################
        output_filename = f'{accession} - {et} - closeness.png'

        closeness = np.array(list(
            nx.closeness_centrality(graphs[et]).items()
        ))[:,1]

        nx.set_node_attributes(
            graphs[et],
            dict(nx.closeness_centrality(graphs[et])),
            'closeness'
        )

        ### LOAD STRUCTURE
        molid = vmd.molecule.load(
            struct_type='pdb',
            struct_file=pdb_file,
            coord_type='pdb',
            coord_file=pdb_file,
        )

        ### SET DISPLAY
        vmd.display.set(
            antialias=True,
            depthcue=False,
            culling=True,
            ambientocclusion=True,
            projection='Orthographic',
            size=(vmd_resolution, vmd_resolution),
            aoambient=0.75,
            aodirect=0.75,
            shadows=False,
            dof=False
        )
        vmd.axes.set_location('OFF')

        ### DRAW VERTICES (RESIDUES)
        backbone = vmd.atomsel('name CA')
        coords = np.vstack([
            backbone.x,
            backbone.y,
            backbone.z,
        ]).T

        n_vertex_colors = 15
        bin_number = np.digitize(
            closeness,
            np.linspace(
                0,
                closeness.max() if closeness.max() > 0 else 1,
                n_vertex_colors,
                endpoint=False
            )
        )
        bin_color = v_cmap( np.linspace(0,1,n_vertex_colors+1)[1:] )[:,:3]

        for color_idx in range(n_vertex_colors):
            bin_idx = color_idx + 1
            coords_subset = coords[bin_number == bin_idx]
            vmd.color.set_colorid(color_idx, tuple(bin_color[color_idx]))
            vmd.graphics.color(molid, color_idx)

            for coord in coords_subset:
                vmd.graphics.sphere(molid, coord, radius=1, resolution=6)

        ### MODIFY MOLECULE REPRESENTATION
        next_color_idx = 0 + n_vertex_colors
        vmd.color.set_colorid(next_color_idx, (1.,1.,1.))
        vmd.graphics.color(molid, next_color_idx)
        if not vmd.molrep.modrep(
            molid,
            0,
            style='NewCartoon',
            color=f'ColorID {next_color_idx}',
            # selection='backbone',
            material='Opaque',
        ): continue

        # vmd.trans.rotate_scene(axis='z', angle=180)
        # vmd.trans.rotate_scene(axis='y', angle=180)

        ### RENDER
        vmd.display.update()
        vmd.render.render(
            method='POV3',
            filename='tmp.pov',
        )

        cmd = [
            'povray',
            f'+W{vmd_resolution}',
            f'+H{vmd_resolution}',
            f'-Itmp.pov',
            f'-O"{plots_save_dir}/{output_filename}"',
            '+X',
            '+A',
            '+FN',
            '+UA',
        ]
        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )

        ### REMOVE MOLECULE FROM DISPLAY
        vmd.molecule.delete(molid)

        ################################################################
        # COMPUTE LAPLACIAN CENTRALITY AND PLOT WITH VMD
        ################################################################
        output_filename = f'{accession} - {et} - laplacian.png'

        laplacian = np.array(list(
            nx.laplacian_centrality(graphs[et]).items()
        ))[:,1]

        nx.set_node_attributes(
            graphs[et],
            dict(nx.laplacian_centrality(graphs[et])),
            'laplacian'
        )

        ### LOAD STRUCTURE
        molid = vmd.molecule.load(
            struct_type='pdb',
            struct_file=pdb_file,
            coord_type='pdb',
            coord_file=pdb_file,
        )

        ### SET DISPLAY
        vmd.display.set(
            antialias=True,
            depthcue=False,
            culling=True,
            ambientocclusion=True,
            projection='Orthographic',
            size=(vmd_resolution, vmd_resolution),
            aoambient=0.75,
            aodirect=0.75,
            shadows=False,
            dof=False
        )
        vmd.axes.set_location('OFF')

        ### DRAW VERTICES (RESIDUES)
        backbone = vmd.atomsel('name CA')
        coords = np.vstack([
            backbone.x,
            backbone.y,
            backbone.z,
        ]).T

        n_vertex_colors = 15
        bin_number = np.digitize(
            laplacian,
            np.linspace(
                0,
                laplacian.max() if laplacian.max() > 0 else 1,
                n_vertex_colors,
                endpoint=False
            )
        )
        bin_color = v_cmap( np.linspace(0,1,n_vertex_colors+1)[1:] )[:,:3]

        for color_idx in range(n_vertex_colors):
            bin_idx = color_idx + 1
            coords_subset = coords[bin_number == bin_idx]
            vmd.color.set_colorid(color_idx, tuple(bin_color[color_idx]))
            vmd.graphics.color(molid, color_idx)

            for coord in coords_subset:
                vmd.graphics.sphere(molid, coord, radius=1, resolution=6)

        ### MODIFY MOLECULE REPRESENTATION
        next_color_idx = 0 + n_vertex_colors
        vmd.color.set_colorid(next_color_idx, (1.,1.,1.))
        vmd.graphics.color(molid, next_color_idx)
        if not vmd.molrep.modrep(
            molid,
            0,
            style='NewCartoon',
            color=f'ColorID {next_color_idx}',
            # selection='backbone',
            material='Opaque',
        ): continue

        # vmd.trans.rotate_scene(axis='z', angle=180)
        # vmd.trans.rotate_scene(axis='y', angle=180)

        ### RENDER
        vmd.display.update()
        vmd.render.render(
            method='POV3',
            filename='tmp.pov',
        )

        cmd = [
            'povray',
            f'+W{vmd_resolution}',
            f'+H{vmd_resolution}',
            f'-Itmp.pov',
            f'-O"{plots_save_dir}/{output_filename}"',
            '+X',
            '+A',
            '+FN',
            '+UA',
        ]
        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )

        ### REMOVE MOLECULE FROM DISPLAY
        vmd.molecule.delete(molid)


Info) Using plugin pdb for structure file /Volumes/Macintosh HD/Dropbox/projects/protein_mechanics-gnn-collagen-oi/ai-Tm_prediction/Geometric-Tm/data/external/AlphaFoldDB/pdb/O31775-AFv4.pdb
Info) Using plugin pdb for coordinates from file /Volumes/Macintosh HD/Dropbox/projects/protein_mechanics-gnn-collagen-oi/ai-Tm_prediction/Geometric-Tm/data/external/AlphaFoldDB/pdb/O31775-AFv4.pdb
Info) Determining bond structure from distance search ...
Info) Finished with coordinate file /Volumes/Macintosh HD/Dropbox/projects/protein_mechanics-gnn-collagen-oi/ai-Tm_prediction/Geometric-Tm/data/external/AlphaFoldDB/pdb/O31775-AFv4.pdb.
Info) Analyzing structure ...
Info)    Atoms: 2067
Info)    Bonds: 2113
Info)    Angles: 0  Dihedrals: 0  Impropers: 0  Cross-terms: 0
Info)    Bondtypes: 0  Angletypes: 0  Dihedraltypes: 0  Impropertypes: 0
Info)    Residues: 264
Info)    Waters: 0
Info)    Segments: 1
Info)    Fragments: 1   Protein: 1   Nucleic: 0
Info) Using plugin pdb for coordinates from file